In [62]:
import pandas as pd
import numpy as np
import re

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

seed = 20191194

In [2]:
df = pd.read_csv('case_study_data_ver1.csv', sep = ';')

In [3]:
df = df.dropna() # getting rid of all NULLs

In [4]:
df['QTY'] = df['QTY'].astype(str)
outtakes = df[df.QTY.str.contains('[a-zA-Z]|\.\d\d$|\.\d\d\d\d$')]
df = df.drop(outtakes.index)
df['QTY'] = df['QTY'].apply(lambda x: re.sub('\.','',re.sub('\..$','',x))).astype(float)
df = df.drop([key for key in list(df.keys()) if df[key].sum()==0], axis=1)

In [5]:
df = df[(df.STORE_ID==3)&(df.ITEM_ID==4)]

In [45]:
# add additional features last x days (This step costs data, critical when there isnt much)
# here we could distinguish: if x <= 5% of data points, use additional features, else not
last_x_days = 7
prev_qty = {}
for num in range(1,last_x_days+1):
    prev_qty['qty_prev_{}d'.format(num)] = [df.QTY.values[i-num] for i in range(len(df))]
    
additional_features = pd.DataFrame(prev_qty)
additional_features.head()

,qty_prev_1d,qty_prev_2d,qty_prev_3d,qty_prev_4d,qty_prev_5d,qty_prev_6d,qty_prev_7d
0,10.0,48.0,21.0,7.0,12.0,7.0,6.0
1,20.0,10.0,48.0,21.0,7.0,12.0,7.0
2,7.0,20.0,10.0,48.0,21.0,7.0,12.0
3,4.0,7.0,20.0,10.0,48.0,21.0,7.0
4,8.0,4.0,7.0,20.0,10.0,48.0,21.0


In [47]:
#merge it
for key in list(additional_features.keys()):
    df['{}'.format(key)]=additional_features['{}'.format(key)].values

df.head()

,ITEM_ID,DATE_ID,STORE_ID,CAT_DAYOFWEEK,DUM_FREEWEEKDAY,DUM_WORKSATURDAY,DUM_SPRINGBREAK,DUM_SUMMERBREAK,DUM_FALLBREAK,DUM_WINTERBREAK,...,QTY,PRICE,CUSTOMER,qty_prev_1d,qty_prev_2d,qty_prev_3d,qty_prev_4d,qty_prev_5d,qty_prev_6d,qty_prev_7d
97,4.0,1.0,3.0,5,0,0,0,0,0,1,...,20.0,1101.45,1607.0,10.0,48.0,21.0,7.0,12.0,7.0,6.0
681,4.0,3.0,3.0,0,0,0,0,0,0,0,...,7.0,1101.45,1498.0,20.0,10.0,48.0,21.0,7.0,12.0,7.0
1192,4.0,4.0,3.0,1,0,0,0,0,0,0,...,4.0,1101.45,1378.0,7.0,20.0,10.0,48.0,21.0,7.0,12.0
1790,4.0,5.0,3.0,2,0,0,0,0,0,0,...,8.0,1101.45,1304.0,4.0,7.0,20.0,10.0,48.0,21.0,7.0
2431,4.0,6.0,3.0,3,0,0,0,0,0,0,...,7.0,1101.45,1538.0,8.0,4.0,7.0,20.0,10.0,48.0,21.0


In [55]:
# Exlude first x days since additional features are actually showing the last days^^
df=df.drop(df[:last_x_days].index)

1149
1142


In [56]:
# Exclude: 
# - ID's, 
# - what we can't know at prediction time 
features = [col for col in list(df.keys()) if (col not in list(df.keys())[:3]) & \
                                              (col not in ['QTY', 'CUSTOMER'])]

In [57]:
# Split training & new data (7 days)
df_validation = df[df.DATE_ID >= df.DATE_ID.max()-7]
df_training = df.drop(df_validation.index)

In [ ]:
std_scale = preprocessing.StandardScaler().fit(df_training[features])

In [63]:
X,y = shuffle(df_training, df_training.QTY, random_state = seed) #np.log(df.QTY)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = seed)

# df_training_std = std_scale.transform(df_training[features])

for data_set in data_sets:
    apply_scaling

In [61]:
base_model_rf = RandomForestRegressor()
base_model_rf.fit(X_train[features],y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)